In [1]:
import torch
from torch._tensor import Tensor
print(torch.__version__)

1.12.1


In [2]:
print(torch.cuda.is_available())

True


In [3]:
print(torch.cuda.get_device_name())

NVIDIA GeForce GTX 1050 Ti


#### Named Tensors
Sometimes we need to index the dimensions of the tensors, so we need to remember the ordering of the dimensions accordingly. But as the data is transfomred this tracking of the dimensions is error prone.\
To keep track of the the indexing of the dimensions, there is a technique called as named tensors, which gives names to the dimensions of the tensor such as an image of 3 channels "RGB" can be (3,5,5) can be given names as Tensor(data = [<A (3,5,5) image data>], name = ["channels","rows", "columns"])

In [4]:
# Lets take make 3 tensors
img_t = torch.randn(3,5,5) # shape[channels, rows, columns]
weights = torch.tensor([0.2126, 0.7152, 0.0722])
batch_t = torch.randn(2,3,5,5) # shape [batch, channels, rows, columns]

Now sometimes the channels of an image are in dimensions 0 i.e when(3,5,5) and sometimes there are batches of images then the channels shift to dimension 1 of the tensor (2,3,5,5) because the 2 at dimension zero is a batch of 2 images.\
But we want our code to generalize so we can see that in both the initializations the channels are at dimension "-3" i.e third from the last dimension. So we will index by -3 to make the indexing compatible for all the initializations.

In [5]:
img_gray_naive = img_t.mean(-3)
batch_gray_naive = batch_t.mean(-3)
print(img_gray_naive.shape, batch_gray_naive.shape)

torch.Size([5, 5]) torch.Size([2, 5, 5])


Pytorch allows us to multiply tensors which have the same shape, as well as shapes which have a operand size is 1 in a given dimension. It also appends leading dimensions of size 1 automatically. This feature is called as broadcasting. The batch_t of shape (2,3,5,5) is multiplied by unsqueezed_weights if shape (3,1,1) and results tensors in the shape of (2,3,5,5), from which we can then sum the third dimension from the end(three channels).

In [6]:
unsqueezed_weights = weights.unsqueeze(-1).unsqueeze(-1)
print(unsqueezed_weights)

tensor([[[0.2126]],

        [[0.7152]],

        [[0.0722]]])


In [7]:
img_weighted = (img_t*unsqueezed_weights)
batch_weighted = batch_t * unsqueezed_weights
img_gray_weigted = img_weighted.sum(-3)
batch_gray_weighted = batch_weighted.sum(-3)
print(batch_weighted.shape, batch_t.shape )

torch.Size([2, 3, 5, 5]) torch.Size([2, 3, 5, 5])


Things can get messy very quickly when we are dealing with tensors of more than 3 dimensions, because we can loose track of the dimensions and what they represented. For eg. (3,5,5) represents an image with 3 channels, 5 rows and 5 columns. So we give names to the dimensions to remember things easily. `This is done by NAMED TENSORS`. Just pass a list of names to the names attribute when creating a tensor to specify the dimensions starting from left to right.
```python
    weights_named = torch.tensor([0.212, 0.414, 0.416], names = ['channels'])
    
```

In [8]:
weights_named = torch.tensor([0.212, 0.414, 0.416], names = ['channels'])

C:\Users\Anant\AppData\Local\Temp\ipykernel_7184\223732023.py:1: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  C:\cb\pytorch_1000000000000\work\c10/core/TensorImpl.h:1408.)
  weights_named = torch.tensor([0.212, 0.414, 0.416], names = ['channels'])


When we want to rename the names given to the dimensions we use the `refine_names()` method on it. `We can use ellipses " ... " to omit any dimension we do not want to add names for`. with `rename` method we can overwrite or drop existing names by passing None in the names list.

In [9]:
image_named = img_t.refine_names(..., "channels", "rows", "columns")
batch_named = batch_t.refine_names(..., "channels",'rows','columns')

print((image_named.shape, image_named.names))
print((batch_named.shape, batch_named.names))


(torch.Size([3, 5, 5]), ('channels', 'rows', 'columns'))
(torch.Size([2, 3, 5, 5]), (None, 'channels', 'rows', 'columns'))


We also have a method known as `align_as()` which would align the dimensions and their names according to the reference tensor that we pass. It returns a tensor with the missing dimensions added and the existing ones permuted in the right order.

In [10]:
print(weights_named.shape)
weights_aligned = weights_named.align_as(image_named)
weights_aligned.shape, weights_aligned.names

torch.Size([3])


(torch.Size([3, 1, 1]), ('channels', 'rows', 'columns'))

Functions accepting dimensions as arguments such as `sum()` also accept names as arguments.

In [11]:
gray_named = (image_named * weights_aligned).sum('channels')
gray_named.shape, gray_named.names

(torch.Size([5, 5]), ('rows', 'columns'))

If we want to use tensors with functions that operate outside of named tensors then we would have to drop the names of the tensor by renaming them to None.

In [12]:
plain_gray = gray_named.rename(None)
plain_gray.shape, plain_gray.names

(torch.Size([5, 5]), (None, None))

NAMED TENSORS ARE AN EXPERIMENTAL FETURE WITH WIDE-SPREAD ADOPTION NOT COMMON, WE WILL STICK TO UNNAMED TENSORS FOR NOW

#### Tensor element types
Standard python numeric types are suboptimal for efficiency because.
1. Python stores every type of data as object. So numeric type data is also stored as a python object with reference counting which is passable for small datapoints but becomes inefficient for a large number of datapoints.
2. A list is python is also a collection of python objects. So this is also a memory inefficient data structure. Moreover this is also a 1D data structure, we can define a list of lists in python to define a 2D or a n-D data structure but that is very inefficient in terms of memory and speed.
3. Python is an iterpreted language so code runtime is slow as compared to other compiled languages like C or C++. For this reason python relies on low-level C code with wrappers around them for ease of use like the numpy library to simulate multidimesional arrays and fast operations on those arrays.\
\
For this reason pytorch has tensors which have all numbers of the same type and pytorch keeps track of this numeric type.\
`The default data type for tensors is 32-bit floating point number.`

#### The Tensor API
A vast majority of methods that are available in the torch module is also available in the tensor module and can be called on a Tensor. For eg. `python torch.transpose()` can be `tensor.transpose()`

##### Tensor : An indexable view for storage
The tensor is an indexable view for the storage instance in the moemory.
1. A `storage` is a `continuous 1D block` of memory. 
2. The `tensor` is stored as a contiguous block of memory and it is managed by `torch.storage instances`. A `storage` is a contiguos block of memory storing numbers of any given type.
3. A `pytorch tensor` is a `view` of storage instances that is capable of indexing into the storage using an `offset` and a `per dimension stride`.
4. Multiple tensors can index from a single storage instance. If a storage instance is defined for a bigger tensor, then a smaller tensor can also index from that using a different offset and dimension stride. This way there is no need to allocate memory for the smaller tensor and retrieving the tensor and performing operations on that becomes very fast.
5. The storage for a given tensor can be accessed by the .storage() property of that tensor.
```python 
    points = torch.tensor([[4.0,3.0], [5.0,2.0], [1.0,2.0]])
    points.storage()
```
6. The layout of the storage is always 1D regardless of the Tensor shape that indexes it(It can be 2D, 3D, 4D .... ND).
7. Changing the value of the storage changes the tensor indexing the storage also.

##### Modifying Stored values : Inplace operations in tensors.
These operations specific to the tensor class and are used to modify the tensors in place rather than returning the other tensors.
1. These methods end with a `trailing undesrscore` like `zero_()`. The zero_() method zeros out all elements of the input. Any method of the tensor class  `without` the trailing underscore will return a new tensor as an output.

In [13]:
points = torch.tensor([[1.0,2.0],[3.0,2.0],[4.0,5.0]], dtype = torch.float32)
points.storage()

 1.0
 2.0
 3.0
 2.0
 4.0
 5.0
[torch.storage._TypedStorage(dtype=torch.float32, device=cpu) of size 6]

In [14]:
a = torch.ones(2,3)
print(a)
print(a.zero_())

tensor([[1., 1., 1.],
        [1., 1., 1.]])
tensor([[0., 0., 0.],
        [0., 0., 0.]])


##### Tensor metadata : Size, offset and Stride
In order to index the storage to return a view known as the tensor, tensors need a size, offset and stride. These three things along with the storage define a tensor.
1. The `size` of the tensor (shape in numpy) is a tuple indicating `how many values in each dimension the tensor represents`.
2. The `offset` of a tensor is the `index in the storage` 1D array which represents the `1st element of the tensor`.
3. The `stride` is the `number of elements that need to be skipped over in the storage to obtain the next element along each dimension in the tensor`. The stride is a tuple denoting the number of elements in storage that need to be skipped when when the index in that particular dimension is increased by 1.
    1. Accessing the `element (i,j)` in tensor results in `storage_offset + stride[0] * i + stride[1] * j` in the storage.
    2. The offset will generally be 0 but if the tensor is generated from the storage of a larger tensor then this will be a positive value.
    3. This indirection between the storage and tensor makes operations inexpensive like transposing a tensor. Instead of doing a memory reallocation we just allocate another object with a different size, offset and stride in most cases.
    4. Changing the subtensor will also change the original storage values. To sidestep this problem we would clone the tensor into a new tensor.

In [15]:
second_point = points[1]
print(second_point.shape)
print(second_point.storage_offset())
print(second_point.stride())

torch.Size([2])
2
(1,)


In [16]:
# Lets now transpose the tensor using the .t() function 
points_tensor = torch.tensor([[1.0,2.0],[2.0,3.0],[5.0,4.0]])
print(f"original: {points_tensor}")

points_t = points_tensor.t() # transposing the tensor
print(f"transposed: {points_t}")

original: tensor([[1., 2.],
        [2., 3.],
        [5., 4.]])
transposed: tensor([[1., 2., 5.],
        [2., 3., 4.]])


In [17]:
# We can easily verify that both the tensors share the same storage.
print(id(points_tensor.storage()) == id(points_t.storage()))

# But they have different strides and shapes and offsets.
print(points_tensor.stride(), points_t.stride())
print(points_tensor.shape, points_t.shape)
print(points_tensor.storage_offset(), points_t.storage_offset())

True
(2, 1) (1, 2)
torch.Size([3, 2]) torch.Size([2, 3])
0 0


In [18]:
# To transpose in higher dimensions we need to provide the dimensions that need to be transposed
t = torch.ones(3,4,2)
print(t.shape)
new_t = t.transpose(0,2)  # 0 is 0th dimension and 2 is the 2nd dimension
print(new_t.shape)

print(t.stride())
print(new_t.stride())

torch.Size([3, 4, 2])
torch.Size([2, 4, 3])
(8, 2, 1)
(1, 2, 8)


Some operations on tensors will only work on contiguous tensors. For eg. `view` works only on contiguous tensors. If we do not have a contiguous tensor then we will get an error and we will have to call the `contiguous()` method to make the tensor contigous. In our case the tensor t is contiugous but new_t which is a transpose of t is not.

In [19]:
print(t.is_contiguous())
print(new_t.is_contiguous())

# we can obtain a contiguous tensor but the stride will change and the storage will also change.
new_t = new_t.contiguous()
print(new_t.is_contiguous())
print(new_t.stride())
print(new_t.storage())
print(new_t.storage_offset())

True
False
True
(12, 3, 1)
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
[torch.storage._TypedStorage(dtype=torch.float32, device=cpu) of size 24]
0


#### Moving tensors to the GPU
If a tensor is created or transferred to the GPU then all the calculations will be carrried out using the GPU routines.
1. To create a tensor on the GPU we can pass a `device='cuda'` argument in the constructor of the tensor. This would create the tensor in GPU memory.
2. We can also copy a tensor created on the CPU to the GPU using the tensor.to() method.

In [20]:
# This tensor will be created on the GPU
points_gpu = torch.tensor([[1.0,2.0], [2.0,3.0], [3.0,4.0]], device='cuda')
print(f"The device of points_gpu is : {points_gpu.device.type}")

# This tensor will be created on the CPU
points_cpu =  torch.tensor([[1.0,2.0], [2.0,3.0], [3.0,4.0]])
print(f"The device of points_cpu is : {points_cpu.device.type}")

# transferring point_cpu to be used on the GPU
points_cpu_transfer = points_cpu.to(device='cuda')
print(f"The device of points_cpu after teansfer is : {points_cpu_transfer.device.type}")

# If our device has more than 1 GPU we can specify which GPU to use with a number starting from zero
p_gpu  = points_cpu.to(device='cuda:0')
print(p_gpu.device.type)

The device of points_gpu is : cuda
The device of points_cpu is : cpu
The device of points_cpu after teansfer is : cuda
cuda


In [21]:
mul_points_gpu = 2 * points_cpu.to(device = 'cuda')
print(mul_points_gpu)

tensor([[2., 4.],
        [4., 6.],
        [6., 8.]], device='cuda:0')


When we perform an operation on the GPU tensor. The result is not copied back to the CPU, the steps that are performed are.
1. The points_cpu tensor is copied to the GPU.
2. The multiplication operation is performed and the result is computed and stored in a new tensor on the GPU.
3. A handle to that GPU tensor is returned. Therefore even if we perform a addition opearation like
     ```python
        mul_points_gpu = mul_points_gpu + 4
    ```
    Then also the calculation is performed on the GPU and no infomration flows to the CPU (unless we print or access the resulting tensor)
4. If we want to move the tensor back to the CPU we would need to provide a `device = 'cpu'` argument in the tensor.to() method.

In [22]:
mul_points_gpu = mul_points_gpu + 4  # Performing the addition(it is still performed on the GPU)
print(mul_points_gpu)               # This operation brings the tensor back to the CPU to print but only a handle which reads from GPU and prints.
print(mul_points_gpu.device.type)
mul_points_cpu = mul_points_gpu.to(device = 'cpu') # This operation moves the tensor back to the CPU
print(mul_points_cpu.device.type)


tensor([[ 6.,  8.],
        [ 8., 10.],
        [10., 12.]], device='cuda:0')
cuda
cpu


#### Numpy interoperability
As numpy is `lingua franca` of the data science world. Pytorch tensors have the ability to convert a tensor to be used as a numpy array and vice versa. We can convert a pytorch tensor to a numpy array without copying the data to a seperate memory location because the storage of the tensor data works with the python buffer protocol.
1. To convert a pytorch tensor to a numpy array we just call the `tensor.numpy()` method.
2. One thing to note here is that the numpy array shares the same buffer as the tensor storage which means that conversion can be done at no cost as long as the data sits in the CPU RAM. This also means that modifying the array will also change the tensor and vice-versa.
3. If the tensor is allocated on the GPU, pytorch will make a copy of the content of the tensor into a numpy array on the CPU as numpy arrays  are not compatible with a GPU.
4. We can obtain a tensor from a numpy array by  torch.from_numpy() method. This would convert the default float64 data of numpy to float32 data of a tensor.

In [23]:
n_tensor = torch.ones(2,3)
print(n_tensor)
n_tensor_numpy = n_tensor.numpy()
print(n_tensor_numpy)
n_tensor_back = torch.from_numpy(n_tensor_numpy)
print(n_tensor_back)

tensor([[1., 1., 1.],
        [1., 1., 1.]])
[[1. 1. 1.]
 [1. 1. 1.]]
tensor([[1., 1., 1.],
        [1., 1., 1.]])


##### Pytorch Dispatcher
Pytorch supports number of tensor types:
1. For GPU's
2. For CPU's
3. Sparse tensors are represented and stored in different ways than Dense tensors.
4. Quantized tensors.
These types are used seamlessly on GPU's or CPU's thorugh a dispatching mechanism by the dispatcher so that the user does not have to use different API's for doing computations on CPU's and GPU's diffferently. Only one can be used for both the CPU's and GPU's.

#### Serializing Tensors : Storing and loading tensors from disk
1. Pytorch can serialize the tensors to save them on disk for saving and loading of the model and saving the weights. Pytorch uses pickle under the hood to serialize the tensors, plus a dedicated serialization code for the storage. by `torch.save()` and `torch.load()`. It can also be used with a `file-reader context manager` of python.
2. But this stored format is compatible for loading only with pytorch. If we want to read the saved tensors with other software then we would need to store them with `HDF5 format`. One of the advantages of the HDF5 format is that we can we can index the data on the disk and only load the data partially. Foe e.g if we wanted to load the last two examples in a dataset of 50000 images we can do that. The data stays on the disk until we ask for the last two images. Then it gets only that data and loads it.